In [1]:
import sqlite3

# Подключение к базе данных (или создание новой, если она не существует)
conn = sqlite3.connect("users.db")
c = conn.cursor()

# Создание таблицы, если она еще не создана
c.execute(
    """
CREATE TABLE IF NOT EXISTS users (
    chat_id INTEGER PRIMARY KEY,
    username TEXT,
    password TEXT
)
"""
)
conn.commit()
conn.close()

In [3]:
import telebot
import sqlite3

# Токен, который вы получили от BotFather
TOKEN = "6898801075:AAGFGEaPfukiQAOGGmh3cbyakKsgEoL3NQU"
bot = telebot.TeleBot(TOKEN)

# Словарь для временного хранения данных от пользователей
user_data = {}


# Обработчик команды start
@bot.message_handler(commands=["start"])
def send_welcome(message):
    bot.reply_to(message, "Привет! Для регистрации введите ваш логин.")
    bot.register_next_step_handler(message, process_username_step)


def process_username_step(message):
    user_id = message.from_user.id
    user_data[user_id] = {"chat_id": user_id}
    user_data[user_id]["username"] = message.text
    bot.reply_to(message, "Теперь введите ваш пароль.")
    bot.register_next_step_handler(message, process_password_step)


def process_password_step(message):
    user_id = message.from_user.id
    user_data[user_id]["password"] = message.text

    # Сохранение данных в базе данных
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute(
        "INSERT INTO users (chat_id, username, password) VALUES (?, ?, ?)",
        (
            user_data[user_id]["chat_id"],
            user_data[user_id]["username"],
            user_data[user_id]["password"],
        ),
    )
    conn.commit()
    conn.close()

    bot.send_message(message.chat.id, "Вы успешно зарегистрированы!")


bot.polling()

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://oldmy.sdu.edu.kz/index.php"
data = {
    "username": "210109032",
    "password": "Alisher_2004",
    "modstring": "",
    "LogIn": " Log in ",
}

response = requests.post(url, data=data, verify=False)
soup = BeautifulSoup(response.text, "html.parser")

# Находим все элементы <td> с классом "clsTdInfo" и стилем "color:#333"
td_elements = soup.find_all("td", class_="clsTdInfo", style="color:#333")

# Извлекаем данные из первых пяти найденных элементов
name_student = td_elements[1].text.strip()
name_advisor = td_elements[3].text.strip()
majority_student = td_elements[5].text.strip()

print("Name of the student:", name_student)
print("Name of the advisor:", name_advisor)
print("Majority of the student:", majority_student)

c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Name of the student: Alisher Kaliyev
Name of the advisor: Думан Тельман
Majority of the student: Математикалық және компьютерлік модельдеу-EN / 2021


In [1]:
import telebot
import sqlite3
from telebot import types
import requests
from bs4 import BeautifulSoup

TOKEN = "6898801075:AAGFGEaPfukiQAOGGmh3cbyakKsgEoL3NQU"
bot = telebot.TeleBot(TOKEN)
user_data = {}

# Initialize the database and create a table if it doesn't exist
conn = sqlite3.connect("users.db", check_same_thread=False)
c = conn.cursor()
c.execute(
    """
CREATE TABLE IF NOT EXISTS users (
    chat_id INTEGER PRIMARY KEY,
    username TEXT,
    password TEXT,
    name_student TEXT,
    name_advisor TEXT,
    majority_student TEXT
)
"""
)
conn.commit()


@bot.message_handler(commands=["start"])
def send_welcome(message):
    chat_id = message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        main_page(chat_id)
    else:
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton(text="Вход", callback_data="login"))
        bot.send_message(chat_id, "Выберите действие:", reply_markup=markup)


@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    user_id = call.from_user.id

    if call.data == "login":
        handle_login_callback(call)
    elif call.data in ["confirm", "cancel"]:
        if user_data.get(user_id):
            if call.data == "confirm":
                confirm_registration(call, user_id)
            elif call.data == "cancel":
                cancel_registration(call, user_id)
        else:
            bot.answer_callback_query(call.id, "Session expired, please start again.")
    elif call.data == "profile":
        handle_profile_callback(call)
    elif call.data == "main_page":
        handle_main_page_callback(call)


def handle_login_callback(call):
    bot.reply_to(call.message, "Введите ваш логин для регистрации.")
    bot.register_next_step_handler(call.message, process_username_step)


def process_username_step(message):
    user_id = message.from_user.id
    user_data[user_id] = {"chat_id": user_id, "username": message.text}
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username = ?", (message.text,))
    existing_user = c.fetchone()

    if existing_user:
        bot.send_message(user_id, "Пользователь с таким именем уже существует!")
        send_welcome(message)
    else:
        bot.reply_to(message, "Теперь введите ваш пароль.")
        bot.register_next_step_handler(message, process_password_step)


def process_password_step(message):
    user_id = message.from_user.id
    user_data[user_id]["password"] = message.text
    url = "https://oldmy.sdu.edu.kz/index.php"
    data = {
        "username": user_data[user_id]["username"],
        "password": message.text,
        "modstring": "",
        "LogIn": " Log in ",
    }

    response = requests.post(url, data=data, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    if "incorrect" in soup.text.lower():
        bot.send_message(
            user_id, "Неправильный пароль. Пожалуйста, попробуйте еще раз."
        )
        send_welcome(message)
    else:
        td_elements = soup.find_all("td", class_="clsTdInfo", style="color:#333")
        user_data[user_id]["name_student"] = td_elements[1].text.strip()
        user_data[user_id]["name_advisor"] = td_elements[3].text.strip()
        user_data[user_id]["majority_student"] = td_elements[5].text.strip()
        keyboard = types.InlineKeyboardMarkup()
        keyboard.row(
            types.InlineKeyboardButton("✅", callback_data="confirm"),
            types.InlineKeyboardButton("❌", callback_data="cancel"),
        )
        bot.send_message(
            message.chat.id,
            f"Подтвердите ваши данные:\nИмя: {user_data[user_id]['name_student']}",
            reply_markup=keyboard,
        )

    login_url = "https://oldmy.sdu.edu.kz/index.php"

    # Perform login request to get cookies
    session = requests.Session()
    login_response = session.post(login_url, data=data, verify=False)

    # Check if login was successful
    if login_response.status_code == 200:
        print("Login successful.")
    else:
        print("Login failed.")
        exit()

    # URL for fetching transcript data
    transcript_url = "https://oldmy.sdu.edu.kz/index.php?mod=transkript"

    # Fetch transcript data using the established session (with cookies)
    transcript_response = session.get(transcript_url)

    # Parse the response HTML using BeautifulSoup
    soup = BeautifulSoup(transcript_response.text, "html.parser")
    print(soup)


def confirm_registration(call, user_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute(
        "INSERT INTO users (chat_id, username, password, name_student, name_advisor, majority_student) VALUES (?, ?, ?, ?, ?, ?)",
        (
            user_data[user_id]["chat_id"],
            user_data[user_id]["username"],
            user_data[user_id]["password"],
            user_data[user_id]["name_student"],
            user_data[user_id]["name_advisor"],
            user_data[user_id]["majority_student"],
        ),
    )
    conn.commit()
    bot.send_message(user_id, "Данные успешно сохранены!")
    main_page(call.message.chat.id)


def cancel_registration(call, user_id):
    bot.send_message(user_id, "Пожалуйста, повторите ввод данных.")
    send_welcome(call.message)


def get_student_data_by_chat_id(chat_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE chat_id = ?", (chat_id,))
    student_data = c.fetchone()
    return student_data


def handle_profile_callback(call):
    chat_id = call.message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(
            types.InlineKeyboardButton("Главная страница", callback_data="main_page")
        )
        bot.send_message(
            chat_id,
            f"Username: {student_data[1]}\nPassword: {student_data[2]}\nName Student: {student_data[3]}\nName Advisor: {student_data[4]}\nMajority Student: {student_data[5]}",
            reply_markup=keyboard,
        )
    else:
        bot.send_message(chat_id, "У вас нет сохраненных данных профиля.")


def handle_main_page_callback(call):
    main_page(call.message.chat.id)


def main_page(chat_id):
    student_data = get_student_data_by_chat_id(chat_id)
    keyboard = types.InlineKeyboardMarkup()
    keyboard.add(types.InlineKeyboardButton("Профиль👤", callback_data="profile"))
    keyboard.add(types.InlineKeyboardButton("Расписание🗓", callback_data="schedule"))
    bot.send_message(
        chat_id,
        f"Добро пожаловать {student_data[3]}🙌🏻! Чем я могу вам помочь?😇",
        reply_markup=keyboard,
    )


bot.polling()

c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

Login successful.
﻿
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="pragma"/>
<title>Student Information System</title>
<link href="css/all.css" rel="stylesheet"/>
<link href="images/icons/favicon.ico" rel="icon" type="image/png"/>
<script src="common/general.js" type="text/javascript"></script>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-74095867-6"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-74095867-6');
</script>
</head>
<body style="padding:0; margin:0; font-family:Arial, Helvetica, sans-serif">
<div align="center">
<div style="max-width:960px !important; padding-top:2px">
<table align="center" border="0" style="max-width:960px !important" width="100%">
<tr>
<td colsp

c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [1]:
import sqlite3
from telebot import TeleBot, types
import requests
from bs4 import BeautifulSoup

# Telegram bot token
TOKEN = "6898801075:AAGFGEaPfukiQAOGGmh3cbyakKsgEoL3NQU"
bot = TeleBot(TOKEN)
user_data = {}

# Initialize the database and create a table if it doesn't exist
conn = sqlite3.connect("users.db", check_same_thread=False)
c = conn.cursor()
# c.execute("DROP TABLE IF EXISTS users")
c.execute(
    """
CREATE TABLE IF NOT EXISTS users (
    chat_id INTEGER PRIMARY KEY,
    student_number TEXT,
    password TEXT,
    name_student TEXT,
    faculty TEXT,
    specialty TEXT,
    level TEXT,
    education_language TEXT,
    entry_date TEXT,
    graduation_date TEXT
)
"""
)
conn.commit()


@bot.message_handler(commands=["start"])
def send_welcome(message):
    chat_id = message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        main_page(chat_id)
    else:
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton(text="Вход", callback_data="login"))
        bot.send_message(chat_id, "Выберите действие:", reply_markup=markup)


@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    if call.data == "login":
        handle_login_callback(call)
    elif call.data in ["confirm", "cancel"]:
        user_id = call.from_user.id
        if user_data.get(user_id):
            if call.data == "confirm":
                confirm_registration(call, user_id)
            elif call.data == "cancel":
                cancel_registration(call, user_id)
        else:
            bot.answer_callback_query(call.id, "Session expired, please start again.")
    elif call.data == "profile":
        handle_profile_callback(call)
    elif call.data == "main_page":
        handle_main_page_callback(call)


def handle_login_callback(call):
    bot.reply_to(call.message, "Введите ваш логин для регистрации.")
    bot.register_next_step_handler(call.message, process_username_step)


def process_username_step(message):
    user_id = message.from_user.id
    user_data[user_id] = {"chat_id": user_id, "student_number": message.text}
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE student_number = ?", (message.text,))
    existing_user = c.fetchone()

    if existing_user:
        bot.send_message(user_id, "Пользователь с таким именем уже существует!")
        send_welcome(message)
    else:
        bot.reply_to(message, "Теперь введите ваш пароль.")
        bot.register_next_step_handler(message, process_password_step)


def process_password_step(message):
    user_id = message.from_user.id
    user_data[user_id]["password"] = message.text
    url = "https://oldmy.sdu.edu.kz/index.php"
    data = {
        "student_number": user_data[user_id]["student_number"],
        "password": user_data[user_id]["password"],
        "modstring": "",  # Modify as per actual form data expected
        "LogIn": " Log in ",
    }
    print(data)

    response = requests.post(url, data=data, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")

    if "incorrect" in soup.text.lower():
        bot.send_message(
            user_id, "Неправильный пароль. Пожалуйста, попробуйте еще раз."
        )
        send_welcome(message)

    # login_url = "https://oldmy.sdu.edu.kz/index.php"

    # # Perform login request to get cookies
    # session = requests.Session()
    # login_response = session.post(login_url, data=data, verify=False)

    # # Check if login was successful
    # if login_response.status_code == 200:
    #     print("Login successful.")
    # else:
    #     print("Login failed.")
    #     exit()

    # # URL for fetching transcript data
    # transcript_url = "https://oldmy.sdu.edu.kz/index.php?mod=transkript"

    # # Fetch transcript data using the established session (with cookies)
    # transcript_response = session.get(transcript_url)

    # # Parse the response HTML using BeautifulSoup
    # soup = BeautifulSoup(transcript_response.text, "html.parser")
    # print(soup)

    else:
        login_url = "https://oldmy.sdu.edu.kz/index.php"

        # Perform login request to get cookies
        session = requests.Session()
        login_response = session.post(login_url, data=data, verify=False)

        # Check if login was successful
        if login_response.status_code == 200:
            print("Login successful.")
        else:
            print("Login failed.")
            exit()

        # URL for fetching transcript data
        transcript_url = "https://oldmy.sdu.edu.kz/index.php?mod=transkript"

        # Fetch transcript data using the established session (with cookies)
        transcript_response = session.get(transcript_url)

        # Parse the response HTML using BeautifulSoup
        soup = BeautifulSoup(transcript_response.text, "html.parser")
        print(soup)
        parse_student_details(soup, user_id, message)


def parse_student_details(soup, user_id, message):
    user_data[user_id]["student_number"] = (
        soup.find("td", text="Student №:").find_next_sibling("td").get_text(strip=True)
    )
    user_data[user_id]["name_student"] = (
        soup.find("td", text="Student Name:")
        .find_next_sibling("td")
        .get_text(strip=True)
    )
    user_data[user_id]["faculty"] = (
        soup.find("td", text="Faculty:").find_next_sibling("td").get_text(strip=True)
    )
    user_data[user_id]["specialty"] = (
        soup.find("td", text="Specialty:").find_next_sibling("td").get_text(strip=True)
    )
    user_data[user_id]["level"] = (
        soup.find("td", text="Level:").find_next_sibling("td").get_text(strip=True)
    )
    user_data[user_id]["education_language"] = (
        soup.find("td", text="Education Language:")
        .find_next_sibling("td")
        .get_text(strip=True)
    )
    user_data[user_id]["entry_date"] = (
        soup.find("td", text="Entry Date:").find_next_sibling("td").get_text(strip=True)
    )
    user_data[user_id]["graduation_date"] = (
        soup.find("td", text="Graduation Date:")
        .find_next_sibling("td")
        .get_text(strip=True)
    )

    keyboard = types.InlineKeyboardMarkup()
    keyboard.row(
        types.InlineKeyboardButton("✅", callback_data="confirm"),
        types.InlineKeyboardButton("❌", callback_data="cancel"),
    )
    bot.send_message(
        message.chat.id,
        f"Подтвердите ваши данные:\nИмя: {user_data[user_id]['name_student']}",
        reply_markup=keyboard,
    )


def confirm_registration(call, user_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute(
        "INSERT INTO users (chat_id, student_number, password, name_student, name_advisor, faculty, specialty, level, education_language, entry_date, graduation_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (
            user_data[user_id]["chat_id"],
            user_data[user_id]["student_number"],
            user_data[user_id]["password"],
            user_data[user_id]["name_student"],
            # "",  # Assuming name_advisor is not fetched in this version
            user_data[user_id]["faculty"],
            user_data[user_id]["specialty"],
            user_data[user_id]["level"],
            user_data[user_id]["education_language"],
            user_data[user_id]["entry_date"],
            user_data[user_id]["graduation_date"],
        ),
    )
    conn.commit()
    bot.send_message(user_id, "Данные успешно сохранены!")
    main_page(call.message.chat.id)


def cancel_registration(call, user_id):
    bot.send_message(user_id, "Пожалуйста, повторите ввод данных.")
    send_welcome(call.message)


def get_student_data_by_chat_id(chat_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE chat_id = ?", (chat_id,))
    student_data = c.fetchone()
    return student_data


def handle_profile_callback(call):
    chat_id = call.message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(
            types.InlineKeyboardButton("Главная страница", callback_data="main_page")
        )
        bot.send_message(
            chat_id,
            f"Username: {student_data[1]}\nPassword: {student_data[2]}\nName Student: {student_data[3]}\nName Advisor: {student_data[4]}\nFaculty: {student_data[5]}",
            reply_markup=keyboard,
        )
    else:
        bot.send_message(chat_id, "У вас нет сохраненных данных профиля.")


def handle_main_page_callback(call):
    main_page(call.message.chat.id)


def main_page(chat_id):
    student_data = get_student_data_by_chat_id(chat_id)
    keyboard = types.InlineKeyboardMarkup()
    keyboard.add(types.InlineKeyboardButton("Профиль👤", callback_data="profile"))
    keyboard.add(types.InlineKeyboardButton("Расписание🗓", callback_data="schedule"))
    bot.send_message(
        chat_id,
        f"Добро пожаловать {student_data[3]}🙌🏻! Чем я могу вам помочь?😇",
        reply_markup=keyboard,
    )


bot.polling()

{'student_number': '210109032', 'password': 'Alisher_2004', 'modstring': '', 'LogIn': ' Log in '}


c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\Honor\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'oldmy.sdu.edu.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

Login successful.
﻿<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Login - Student Information System</title>
<link href="css/loginAuth.css" rel="stylesheet"/>
<link href="images/icons/favicon.ico" rel="icon" type="image/png"/>
<script>
	function OnL() {
		var el = document.getElementById("username");
		if(typeof el == "undefined") return;
		if(el.value != "") {
			var elPsw = document.getElementById("password");
			elPsw.focus();
		}
		else el.focus();
	}
</script>
</head>
<body onload="OnL();">
<div style="position:absolute;width:100%;height:100%;left:0;top:0;overflow:hidden;min-height:700px;padding:0;margin:0;">
<div style="position:relative; text-align:right; top:5pt">
<a href="?lng=kz"><img border="0" height="15" src="images/kz.png" title="Қазақ тілі"/></a> 
<a href="?lng=en"><img b

AttributeError: 'NoneType' object has no attribute 'find_next_sibling'

In [6]:
import telebot
import sqlite3
from telebot import types
import requests
from bs4 import BeautifulSoup

TOKEN = "6898801075:AAGFGEaPfukiQAOGGmh3cbyakKsgEoL3NQU"
bot = telebot.TeleBot(TOKEN)
user_data = {}

# Initialize the database and create a table if it doesn't exist
conn = sqlite3.connect("users.db", check_same_thread=False)
c = conn.cursor()
c.execute(
    """
CREATE TABLE IF NOT EXISTS users (
    chat_id INTEGER PRIMARY KEY,
    username TEXT,
    password TEXT,
    name_student TEXT,
    faculty TEXT,
    specialty TEXT,
    level TEXT,
    education_language TEXT,
    entry_date TEXT,
    graduation_date TEXT
)
"""
)
conn.commit()


@bot.message_handler(commands=["start"])
def send_welcome(message):
    chat_id = message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        main_page(chat_id)
    else:
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton(text="Вход", callback_data="login"))
        bot.send_message(chat_id, "Выберите действие:", reply_markup=markup)


@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    user_id = call.from_user.id

    if call.data == "login":
        handle_login_callback(call)
    elif call.data in ["confirm", "cancel"]:
        if user_data.get(user_id):
            if call.data == "confirm":
                confirm_registration(call, user_id)
            elif call.data == "cancel":
                cancel_registration(call, user_id)
        else:
            bot.answer_callback_query(call.id, "Session expired, please start again.")
    elif call.data == "profile":
        handle_profile_callback(call)
    elif call.data == "main_page":
        handle_main_page_callback(call)


def handle_login_callback(call):
    bot.reply_to(call.message, "Введите ваш логин для регистрации.")
    bot.register_next_step_handler(call.message, process_username_step)


def process_username_step(message):
    user_id = message.from_user.id
    user_data[user_id] = {"chat_id": user_id, "username": message.text}
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username = ?", (message.text,))
    existing_user = c.fetchone()

    if existing_user:
        bot.send_message(user_id, "Пользователь с таким именем уже существует!")
        send_welcome(message)
    else:
        bot.reply_to(message, "Теперь введите ваш пароль.")
        bot.register_next_step_handler(message, process_password_step)


def process_password_step(message):
    user_id = message.from_user.id
    user_data[user_id]["password"] = message.text
    url = "https://oldmy.sdu.edu.kz/index.php"
    data = {
        "username": user_data[user_id]["username"],
        "password": message.text,
        "modstring": "",
        "LogIn": " Log in ",
    }

    response = requests.post(url, data=data, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    if "incorrect" in soup.text.lower():
        bot.send_message(
            user_id, "Неправильный пароль. Пожалуйста, попробуйте еще раз."
        )
        send_welcome(message)
    else:
        # td_elements = soup.find_all("td", class_="clsTdInfo", style="color:#333")
        # user_data[user_id]["name_student"] = td_elements[1].text.strip()
        # user_data[user_id]["name_advisor"] = td_elements[3].text.strip()
        # user_data[user_id]["majority_student"] = td_elements[5].text.strip()

        login_url = "https://oldmy.sdu.edu.kz/index.php"

        # Perform login request to get cookies
        session = requests.Session()
        login_response = session.post(login_url, data=data, verify=False)

        # Check if login was successful
        if login_response.status_code == 200:
            print("Login successful.")
        else:
            print("Login failed.")
            exit()

        # URL for fetching transcript data
        transcript_url = "https://oldmy.sdu.edu.kz/index.php?mod=transkript"

        # Fetch transcript data using the established session (with cookies)
        transcript_response = session.get(transcript_url)

        # Parse the response HTML using BeautifulSoup
        soup = BeautifulSoup(transcript_response.text, "html.parser")

        user_data[user_id]["username"] = (
            soup.find("td", text="Student №:").find_next_sibling("td").get_text(strip=True)
        )
        user_data[user_id]["name_student"] = (
            soup.find("td", text="Student Name:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["faculty"] = (
            soup.find("td", text="Faculty:").find_next_sibling("td").get_text(strip=True)
        )
        user_data[user_id]["specialty"] = (
            soup.find("td", text="Specialty:").find_next_sibling("td").get_text(strip=True)
        )
        user_data[user_id]["level"] = (
            soup.find("td", text="Level:").find_next_sibling("td").get_text(strip=True)
        )
        user_data[user_id]["education_language"] = (
            soup.find("td", text="Education Language:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["entry_date"] = (
            soup.find("td", text="Entry Date:").find_next_sibling("td").get_text(strip=True)
        )
        user_data[user_id]["graduation_date"] = (
            soup.find("td", text="Graduation Date:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        keyboard = types.InlineKeyboardMarkup()
        keyboard.row(
            types.InlineKeyboardButton("✅", callback_data="confirm"),
            types.InlineKeyboardButton("❌", callback_data="cancel"),
        )
        bot.send_message(
            message.chat.id,
            f"Подтвердите ваши данные:\nИмя: {user_data[user_id]['name_student']}",
            reply_markup=keyboard,
        )


def confirm_registration(call, user_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute(
        "INSERT INTO users (chat_id, username, password, name_student, faculty, specialty, level, education_language, entry_date, graduation_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (
            user_data[user_id]["chat_id"],
            user_data[user_id]["username"],
            user_data[user_id]["password"],
            user_data[user_id]["name_student"],
            # user_data[user_id]["majority_student"],
            user_data[user_id]["faculty"],
            user_data[user_id]["specialty"],
            user_data[user_id]["level"],
            user_data[user_id]["education_language"],
            user_data[user_id]["entry_date"],
            user_data[user_id]["graduation_date"],
        ),
    )

    conn.commit()
    bot.send_message(user_id, "Данные успешно сохранены!")
    main_page(call.message.chat.id)


def cancel_registration(call, user_id):
    bot.send_message(user_id, "Пожалуйста, повторите ввод данных.")
    send_welcome(call.message)


def get_student_data_by_chat_id(chat_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE chat_id = ?", (chat_id,))
    student_data = c.fetchone()
    return student_data


def handle_profile_callback(call):
    chat_id = call.message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(
            types.InlineKeyboardButton("Главная страница", callback_data="main_page")
        )
        bot.send_message(
            chat_id,
            f"Student ID: {student_data[1]}\nPassword: {student_data[2]}\nName Student: {student_data[3]}\nFaculty: {student_data[4]}\nSpecialty: {student_data[5]}\nLevel: {student_data[6]}\nEducation Language: {student_data[7]}\nEntry Date: {student_data[8]}\nGraduation Date: {student_data[9]}",
            reply_markup=keyboard,
        )

    else:
        bot.send_message(chat_id, "У вас нет сохраненных данных профиля.")


def handle_main_page_callback(call):
    main_page(call.message.chat.id)


def main_page(chat_id):
    student_data = get_student_data_by_chat_id(chat_id)
    keyboard = types.InlineKeyboardMarkup()
    keyboard.add(types.InlineKeyboardButton("Профиль👤", callback_data="profile"))
    keyboard.add(types.InlineKeyboardButton("Расписание🗓", callback_data="schedule"))
    bot.send_message(
        chat_id,
        f"Добро пожаловать {student_data[3]}🙌🏻! Чем я могу вам помочь?😇",
        reply_markup=keyboard,
    )


bot.polling()

: 

Транскрипт

In [3]:
import telebot
import sqlite3
from telebot import types
import requests
from bs4 import BeautifulSoup

TOKEN = "6898801075:AAGFGEaPfukiQAOGGmh3cbyakKsgEoL3NQU"
bot = telebot.TeleBot(TOKEN)
user_data = {}

# Initialize the database and create a table if it doesn't exist
conn = sqlite3.connect("users.db", check_same_thread=False)
c = conn.cursor()
c.execute(
    """
CREATE TABLE IF NOT EXISTS users (
    chat_id INTEGER PRIMARY KEY,
    username TEXT,
    password TEXT,
    name_student TEXT,
    faculty TEXT,
    specialty TEXT,
    level TEXT,
    education_language TEXT,
    entry_date TEXT,
    graduation_date TEXT
)
"""
)
conn.commit()


@bot.message_handler(commands=["start"])
def send_welcome(message):
    chat_id = message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        main_page(chat_id)
    else:
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton(text="Вход", callback_data="login"))
        bot.send_message(chat_id, "Выберите действие:", reply_markup=markup)


@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    user_id = call.from_user.id

    if call.data == "login":
        handle_login_callback(call)
    elif call.data in ["confirm", "cancel"]:
        if user_data.get(user_id):
            if call.data == "confirm":
                confirm_registration(call, user_id)
            elif call.data == "cancel":
                cancel_registration(call, user_id)
        else:
            bot.answer_callback_query(call.id, "Session expired, please start again.")
    elif call.data == "profile":
        handle_profile_callback(call)
    elif call.data == "main_page":
        handle_main_page_callback(call)


def handle_login_callback(call):
    bot.reply_to(call.message, "Введите ваш логин для регистрации.")
    bot.register_next_step_handler(call.message, process_username_step)


def process_username_step(message):
    user_id = message.from_user.id
    user_data[user_id] = {"chat_id": user_id, "username": message.text}
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username = ?", (message.text,))
    existing_user = c.fetchone()

    if existing_user:
        bot.send_message(user_id, "Пользователь с таким именем уже существует!")
        send_welcome(message)
    else:
        bot.reply_to(message, "Теперь введите ваш пароль.")
        bot.register_next_step_handler(message, process_password_step)


def process_password_step(message):
    user_id = message.from_user.id
    user_data[user_id]["password"] = message.text
    url = "https://oldmy.sdu.edu.kz/index.php"
    data = {
        "username": user_data[user_id]["username"],
        "password": message.text,
        "modstring": "",
        "LogIn": " Log in ",
    }

    response = requests.post(url, data=data, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    if "incorrect" in soup.text.lower():
        bot.send_message(
            user_id, "Неправильный пароль. Пожалуйста, попробуйте еще раз."
        )
        send_welcome(message)
    else:
        # td_elements = soup.find_all("td", class_="clsTdInfo", style="color:#333")
        # user_data[user_id]["name_student"] = td_elements[1].text.strip()
        # user_data[user_id]["name_advisor"] = td_elements[3].text.strip()
        # user_data[user_id]["majority_student"] = td_elements[5].text.strip()

        login_url = "https://oldmy.sdu.edu.kz/index.php"

        # Perform login request to get cookies
        session = requests.Session()
        login_response = session.post(login_url, data=data, verify=False)

        # Check if login was successful
        if login_response.status_code == 200:
            print("Login successful.")
        else:
            print("Login failed.")
            exit()

        # URL for fetching transcript data
        transcript_url = "https://oldmy.sdu.edu.kz/index.php?mod=transkript"

        # Fetch transcript data using the established session (with cookies)
        transcript_response = session.get(transcript_url)

        # Parse the response HTML using BeautifulSoup
        soup = BeautifulSoup(transcript_response.text, "html.parser")

        user_data[user_id]["username"] = (
            soup.find("td", text="Student №:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["name_student"] = (
            soup.find("td", text="Student Name:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["faculty"] = (
            soup.find("td", text="Faculty:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["specialty"] = (
            soup.find("td", text="Specialty:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["level"] = (
            soup.find("td", text="Level:").find_next_sibling("td").get_text(strip=True)
        )
        user_data[user_id]["education_language"] = (
            soup.find("td", text="Education Language:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["entry_date"] = (
            soup.find("td", text="Entry Date:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        user_data[user_id]["graduation_date"] = (
            soup.find("td", text="Graduation Date:")
            .find_next_sibling("td")
            .get_text(strip=True)
        )
        keyboard = types.InlineKeyboardMarkup()
        keyboard.row(
            types.InlineKeyboardButton("✅", callback_data="confirm"),
            types.InlineKeyboardButton("❌", callback_data="cancel"),
        )
        bot.send_message(
            message.chat.id,
            f"Подтвердите ваши данные:\nИмя: {user_data[user_id]['name_student']}",
            reply_markup=keyboard,
        )


def confirm_registration(call, user_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute(
        "INSERT INTO users (chat_id, username, password, name_student, faculty, specialty, level, education_language, entry_date, graduation_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (
            user_data[user_id]["chat_id"],
            user_data[user_id]["username"],
            user_data[user_id]["password"],
            user_data[user_id]["name_student"],
            # user_data[user_id]["majority_student"],
            user_data[user_id]["faculty"],
            user_data[user_id]["specialty"],
            user_data[user_id]["level"],
            user_data[user_id]["education_language"],
            user_data[user_id]["entry_date"],
            user_data[user_id]["graduation_date"],
        ),
    )

    conn.commit()
    bot.send_message(user_id, "Данные успешно сохранены!")
    main_page(call.message.chat.id)


def cancel_registration(call, user_id):
    bot.send_message(user_id, "Пожалуйста, повторите ввод данных.")
    send_welcome(call.message)


def get_student_data_by_chat_id(chat_id):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE chat_id = ?", (chat_id,))
    student_data = c.fetchone()
    return student_data


def handle_profile_callback(call):
    chat_id = call.message.chat.id
    student_data = get_student_data_by_chat_id(chat_id)
    if student_data:
        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(types.InlineKeyboardButton("Go to main page", callback_data="main_page"))
        bot.send_message(
            chat_id,
            f"Student ID: {student_data[1]}\nPassword: {student_data[2]}\nName Student: {student_data[3]}\nFaculty: {student_data[4]}\nSpecialty: {student_data[5]}\nLevel: {student_data[6]}\nEducation Language: {student_data[7]}\nEntry Date: {student_data[8]}\nGraduation Date: {student_data[9]}",
            reply_markup=keyboard,
        )

    else:
        bot.send_message(chat_id, "У вас нет сохраненных данных профиля.")


def handle_main_page_callback(call):
    main_page(call.message.chat.id)


def main_page(chat_id):
    student_data = get_student_data_by_chat_id(chat_id)
    keyboard = types.InlineKeyboardMarkup()
    keyboard.add(types.InlineKeyboardButton("Profile👤", callback_data="profile"))
    keyboard.add(types.InlineKeyboardButton("Schedule🗓", callback_data="schedule"))
    keyboard.add(types.InlineKeyboardButton("Transcript🗂", callback_data="schedule"))
    bot.send_message(
        chat_id,
        f"Добро пожаловать {student_data[3]}🙌🏻! Чем я могу вам помочь?😇",
        reply_markup=keyboard,
    )


bot.polling()